In [1]:
import numpy as np
import cv2 as cv
import json
from math import sqrt, pow

In [2]:
def execute(src, out, minArea=5, drawLabel=True):
    img = cv.imread(src, 0)
    labeled = cv.cvtColor(img, 0, cv.COLOR_GRAY2BGR)
    # External tree or external as param
    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL,  cv.CHAIN_APPROX_NONE)
    # Sort by larger areas
    contoursFeatures = []
    cnts = sorted(contours, key=cv.contourArea, reverse=True)
    for i in range(0, len(cnts)):
        if cv.contourArea(cnts[i]) >= minArea:
            features = {}
            M = cv.moments(cnts[i])
            centroid = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            features['xCentroid'] = centroid[0]
            features['yCentroid'] = centroid[1]
            features['arcLength'] = cv.arcLength(cnts[i], True)
            features['area'] = cv.contourArea(cnts[i])
            if drawLabel:
                cv.putText(labeled, str(i), (centroid[0], centroid[1]), cv.FONT_HERSHEY_SIMPLEX, 1, (209, 80, 0, 255), 3)
            try:
                # Calculate eccentricity
                (x, y), (MA, ma), angle = cv.fitEllipse(cnts[i])
                a = ma/2
                b = MA/2

                if (a > b):
                    eccentricity = sqrt(pow(a, 2)-pow(b, 2))
                    eccentricity = round(eccentricity/a, 2)
                else:
                    eccentricity = sqrt(pow(b, 2)-pow(a, 2))
                    eccentricity = round(eccentricity/b, 2)

                features['eccentricity'] = eccentricity
            except Error:
                features['eccentricity'] = 0
            contoursFeatures.append(features)
    f = open(out, 'w')
    f.write(json.dumps(contoursFeatures))
    if drawLabel:
        cv.imwrite(src.replace('.png', 'labeled.png'), labeled)

In [3]:
execute('./dl1.png', './features.json', 5)